In [1]:
import os
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf

from sklearn.model_selection import ParameterGrid
from time import time

# Generate data

In [2]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lorenz_data(128, noise_strength=noise_strength)
validation_data = get_lorenz_data(2, noise_strength=noise_strength)

In [3]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3 #
params['model_order'] = 1
params['poly_order'] = 3 #
params['include_sine'] = False #
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0 #
params['loss_weight_sindy_z'] = 0.0 #
params['loss_weight_sindy_x'] = 1e-4 #
params['loss_weight_sindy_regularization'] = 1e-5 #

params['activation'] = 'sigmoid' #
params['widths'] = [64,32] #

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 128 #
params['learning_rate'] = 1e-2 #

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 100
params['refinement_epochs'] = 20


In [4]:
def dftocsv(df):
    if os.path.isfile('gridsearch_results.csv'):
        df.to_csv('gridsearch_results.csv', mode='a', header=False)
    else:
        df.to_csv('gridsearch_results.csv', mode='w', header=True)

# Set up model and training parameters

In [5]:
param_grid = {
    'latent_dim': [1, 3, 6],
    'loss_weight_sindy_z': [0.0, 1e-4],
    'loss_weight_sindy_x': [0.0, 1e-4],
    'widths' : [[64],[64, 32],[64,32,16]],
    'batch_size': [128, 32]
}
grid = ParameterGrid(param_grid)



# Run training experiments

In [6]:
for i, changed_params in enumerate(grid):
    params_run = params.copy()
    params_run.update(changed_params)
    params_run['library_dim'] = library_size(params_run['latent_dim'], params_run['poly_order'], params_run['include_sine'], True)
    params_run['coefficient_mask'] = np.ones((params_run['library_dim'], params_run['latent_dim']))
    
    print(f'EXPERIMENT:{i+1}')
    start = time()

    tf.reset_default_graph()
    results_dict = train_network(training_data, validation_data, params_run)
    end = time()
    dt = end-start

    df = pd.DataFrame()
    df = df.append({**results_dict, **params_run}, ignore_index=True)

    df2save = df[['loss_decoder','loss_sindy_x', 'loss_sindy_z', 'loss_sindy_regularization','validation_losses']]
    changed_params['widths'] = str(changed_params['widths'])
    df_params = pd.DataFrame(changed_params, index=[0])

    df_final = pd.concat([df2save, df_params], axis=1)
    df_final['dt'] = dt

    dftocsv(df_final)

print("DONE")

EXPERIMENT:1


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


EXPERIMENT:2
EXPERIMENT:3
EXPERIMENT:4
EXPERIMENT:5
EXPERIMENT:6
EXPERIMENT:7
EXPERIMENT:8
EXPERIMENT:9
EXPERIMENT:10
EXPERIMENT:11
EXPERIMENT:12
EXPERIMENT:13
EXPERIMENT:14
EXPERIMENT:15
EXPERIMENT:16
EXPERIMENT:17
EXPERIMENT:18
EXPERIMENT:19
EXPERIMENT:20
EXPERIMENT:21
EXPERIMENT:22
EXPERIMENT:23
EXPERIMENT:24
EXPERIMENT:25
EXPERIMENT:26
EXPERIMENT:27
EXPERIMENT:28
EXPERIMENT:29
EXPERIMENT:30
EXPERIMENT:31
EXPERIMENT:32
EXPERIMENT:33
EXPERIMENT:34
EXPERIMENT:35
EXPERIMENT:36
EXPERIMENT:37
EXPERIMENT:38
EXPERIMENT:39
